# Определение возраста покупателей по фотографии

Описание проекта:
Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:
- Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
- Контролировать добросовестность кассиров при продаже алкоголя.
Необходимо построить модель, которая по фотографии определит приблизительный возраст человека. В нашем распоряжении набор фотографий людей с указанием возраста.

План работ: 
- Исследовательский анализ набора фотографий.
- Подготовка данных к обучению.
- Обучение нейронной сети и проверка качества.

## Исследовательский анализ данных

In [1]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf



In [2]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

In [3]:
labels.info()

In [4]:
labels.describe()

In [5]:
display(labels.head())

In [6]:
labels.hist(bins=100,color = 'red')
plt.title("Возраст покупателей")
plt.xlabel('Возраст')
plt.ylabel('Частота наблюдений')
plt.show()

In [7]:
labels.shape

In [8]:
# выгрузим 16 фото
features, target = next(train_gen_flow)
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(top=0.95, wspace=0.5, hspace=0.98)
for i in range(16):
    fig.add_subplot(4, 4, i+1)
    plt.suptitle('Фотографии', fontsize=10)
    plt.title(f'Возраст-{target[i]}', fontsize=10)
    plt.imshow(features[i])
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    

В наборе данных 7591  цветных и черно-белых изображений: на фото люди в основном в возрасте от 20 до 40 лет( примерно).

## Обучение модели



def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25, horizontal_flip=True, rescale=1./255)
    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)

    return train_gen_flow

def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    test_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_gen_flow


def create_model(input_shape):

    backbone = ResNet50(input_shape=input_shape,
                    weights='imagenet', 
                    include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))

    optimizer = Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error',
                  metrics=['mae'])


    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=10,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

356/356 - 63s - loss: 195.8551 - mae: 10.2801 - val_loss: 505.0129 - val_mae: 17.3196
Epoch 2/10
356/356 - 38s - loss: 83.1188 - mae: 6.8878 - val_loss: 130.5519 - val_mae: 8.5350
Epoch 3/10
Epoch 4/10
356/356 - 38s - loss: 58.5470 - mae: 5.8733 - val_loss: 83.4839 - val_mae: 6.8935
356/356 - 38s - loss: 43.7499 - mae: 5.0593 - val_loss: 82.9732 - val_mae: 6.5932
Epoch 5/10
356/356 - 38s - loss: 31.9636 - mae: 4.3218 - val_loss: 129.8647 - val_mae: 8.6651
Epoch 6/10
356/356 - 38s - loss: 25.3340 - mae: 3.8557 - val_loss: 64.8973 - val_mae: 5.9833
Epoch 7/10
356/356 - 38s - loss: 20.5816 - mae: 3.4576 - val_loss: 69.2085 - val_mae: 6.0955
Epoch 8/10
356/356 - 38s - loss: 16.9891 - mae: 3.1337 - val_loss: 74.4964 - val_mae: 6.4068
Epoch 9/10
356/356 - 40s - loss: 14.3631 - mae: 2.9120 - val_loss: 64.8083 - val_mae: 6.0036
Epoch 10/10
356/356 - 41s - loss: 12.9143 - mae: 2.7618 - val_loss: 66.7903 - val_mae: 6.0363
119/119 - 10s - loss: 66.7903 - mae: 6.0363
Test MAE: 6.0363

## Анализ обученной модели

Цели и задачи: Нам требовалось построить и обучить свёрточную нейронную сеть на наборе данных с фотографиями, получить значения MAE на тестовой выборке не больше 8.

Итоги: использование нейронных сетей позволяет эффективно обработать большое количество информации с достаточной точностью.
Была использована модель сверточной нейронной сети,полученная модель вполне может решить поставленную задачу.